# **14.5 File_Paths_and_Directories**

Real Pokemon games organize files into folders — save files in one directory, logs in another, Pokemon sprites somewhere else. Working with file paths and directories is essential for building organized, cross-platform programs. In this lesson you'll learn to build paths safely, navigate directories, check if files exist, and manipulate the file system using Python's `pathlib` and `os` modules.

---

## **The Problem with String Paths**

Windows uses backslashes (`C:\Users\Ash\saves`), while Mac/Linux use forward slashes (`/home/ash/saves`). Hardcoding paths as strings breaks cross-platform compatibility and makes code fragile.

In [ ]:
# BAD: Hardcoded string paths (Windows-specific)
# save_path = "C:\\Users\\Ash\\pokemon\\saves\\team.json"  # Breaks on Mac/Linux

# BETTER: Use os.path.join (cross-platform)
import os
save_path = os.path.join('pokemon', 'saves', 'team.json')
print(f"os.path.join result: {save_path}")

# BEST: Use pathlib (modern, object-oriented)
from pathlib import Path
save_path = Path('pokemon') / 'saves' / 'team.json'
print(f"pathlib result: {save_path}")
print(f"Type: {type(save_path)}")

---

## **Using pathlib.Path (Modern Approach)**

`pathlib.Path` is Python 3's modern way to work with file paths. It's object-oriented, cross-platform, and much cleaner than string manipulation. The `/` operator joins path components automatically.

In [ ]:
from pathlib import Path

# Create paths with / operator — works on all platforms
base = Path('pokemon_game')
saves = base / 'saves'
team_file = saves / 'team.json'

print(f"Base:      {base}")
print(f"Saves dir: {saves}")
print(f"Team file: {team_file}")

# Path components
print(f"\nParent: {team_file.parent}")     # pokemon_game/saves
print(f"Name:   {team_file.name}")         # team.json
print(f"Stem:   {team_file.stem}")         # team (no extension)
print(f"Suffix: {team_file.suffix}")       # .json

# Absolute path
absolute = team_file.absolute()
print(f"\nAbsolute: {absolute}")

---

## **Checking If Files and Directories Exist**

Before trying to read a file or create a directory, check if it already exists. Path objects have `.exists()`, `.is_file()`, and `.is_dir()` methods.

In [ ]:
from pathlib import Path

# Check different path types
paths_to_check = [
    Path('pokemon_stats.csv'),       # File we created earlier
    Path('nonexistent.txt'),          # Doesn't exist
    Path('.'),                        # Current directory
]

for path in paths_to_check:
    print(f"\n{path}:")
    print(f"  exists():  {path.exists()}")
    print(f"  is_file(): {path.is_file()}")
    print(f"  is_dir():  {path.is_dir()}")

# Safe file operations
save_file = Path('my_save.json')

if save_file.exists():
    print(f"\n{save_file} already exists — loading...")
else:
    print(f"\n{save_file} not found — creating new save...")

---

## **Creating Directories**

Use `.mkdir()` to create directories. The `parents=True` argument creates all intermediate directories if they don't exist, and `exist_ok=True` prevents errors if the directory is already there.

In [ ]:
from pathlib import Path

# Create a directory structure for Pokemon game
game_dir = Path('pokemon_game')
saves_dir = game_dir / 'saves'
logs_dir = game_dir / 'logs'
data_dir = game_dir / 'data' / 'pokedex'  # Nested directories

# mkdir with parents=True creates all intermediate directories
saves_dir.mkdir(parents=True, exist_ok=True)
logs_dir.mkdir(parents=True, exist_ok=True)
data_dir.mkdir(parents=True, exist_ok=True)

print("Directories created:")
print(f"  {saves_dir}")
print(f"  {logs_dir}")
print(f"  {data_dir}")

# Verify
print(f"\n{saves_dir} exists: {saves_dir.exists()}")
print(f"{saves_dir} is a directory: {saves_dir.is_dir()}")

---

## **Listing Files in a Directory**

Use `.iterdir()` to list all items in a directory, or `.glob()` to find files matching a pattern. These return iterators of Path objects.

In [ ]:
from pathlib import Path

# Create some sample files
saves_dir = Path('pokemon_game/saves')
saves_dir.mkdir(parents=True, exist_ok=True)

(saves_dir / 'ash_save.json').write_text('{"trainer": "Ash"}')
(saves_dir / 'misty_save.json').write_text('{"trainer": "Misty"}')
(saves_dir / 'brock_save.json').write_text('{"trainer": "Brock"}')
(saves_dir / 'backup.txt').write_text('Backup data')

# List all files
print("All files in saves/:")
for item in saves_dir.iterdir():
    if item.is_file():
        print(f"  {item.name}")

# List only JSON files using glob
print("\nOnly JSON files:")
for json_file in saves_dir.glob('*.json'):
    print(f"  {json_file.name}")

# Recursive glob — find all JSON files in subdirectories too
print("\nAll JSON files (recursive):")
for json_file in Path('pokemon_game').rglob('*.json'):
    print(f"  {json_file}")

---

## **Reading and Writing with Path Objects**

Path objects have convenience methods for quick file I/O: `.read_text()`, `.write_text()`, `.read_bytes()`, and `.write_bytes()`.

In [ ]:
from pathlib import Path

# Create a Path object
config_file = Path('pokemon_game/config.txt')

# Write text in one line
config_file.write_text("difficulty=normal\nvolume=80\nauto_save=true\n")
print(f"Wrote to {config_file}")

# Read text in one line
content = config_file.read_text()
print(f"\nRead from {config_file}:")
print(content)

# Also works with open() for more control
with config_file.open('r') as f:
    for line in f:
        key, value = line.strip().split('=')
        print(f"  {key}: {value}")

---

## **Deleting Files and Directories**

Use `.unlink()` to delete files and `.rmdir()` to delete empty directories. For non-empty directories, use `shutil.rmtree()`.

In [ ]:
from pathlib import Path
import shutil

# Create a temporary file
temp_file = Path('temp_data.txt')
temp_file.write_text('Temporary Pokemon data')
print(f"Created: {temp_file}")
print(f"Exists: {temp_file.exists()}")

# Delete the file
temp_file.unlink()
print(f"\nDeleted: {temp_file}")
print(f"Exists: {temp_file.exists()}")

# Delete an empty directory
empty_dir = Path('empty_folder')
empty_dir.mkdir(exist_ok=True)
print(f"\nCreated: {empty_dir}")
empty_dir.rmdir()  # Only works if empty
print(f"Deleted: {empty_dir}")

# Delete a directory with contents
test_dir = Path('test_folder')
test_dir.mkdir(exist_ok=True)
(test_dir / 'file.txt').write_text('data')
print(f"\nDeleting non-empty directory: {test_dir}")
shutil.rmtree(test_dir)  # Deletes directory and all contents
print(f"Deleted: {test_dir}")

---

## **Getting File Information**

Path objects provide access to file metadata like size, modification time, and permissions through `.stat()`.

In [ ]:
from pathlib import Path
import datetime

# Create a sample file
data_file = Path('pokemon_data.txt')
data_file.write_text('Pikachu,Electric,25\nCharizard,Fire,36\n')

# Get file statistics
stats = data_file.stat()

print(f"File: {data_file}\n")
print(f"Size: {stats.st_size} bytes")
print(f"Modified: {datetime.datetime.fromtimestamp(stats.st_mtime)}")
print(f"Created: {datetime.datetime.fromtimestamp(stats.st_ctime)}")

# Human-readable size
size_bytes = stats.st_size
if size_bytes < 1024:
    print(f"Size: {size_bytes} bytes")
elif size_bytes < 1024 * 1024:
    print(f"Size: {size_bytes / 1024:.1f} KB")
else:
    print(f"Size: {size_bytes / (1024 * 1024):.1f} MB")

---

## **Working Directory and Home Directory**

Python programs have a current working directory. You can get it with `Path.cwd()` and get the user's home directory with `Path.home()`.

In [ ]:
from pathlib import Path

# Current working directory
cwd = Path.cwd()
print(f"Current working directory: {cwd}")

# User's home directory
home = Path.home()
print(f"Home directory: {home}")

# Build a path relative to home
saves_path = home / 'PokemonGame' / 'saves'
print(f"\nSave files would go in: {saves_path}")

# Resolve relative paths to absolute
relative = Path('pokemon_game/saves')
absolute = relative.resolve()
print(f"\nRelative: {relative}")
print(f"Absolute: {absolute}")

---

## **Practical: Organized Game File Structure**

In [ ]:
from pathlib import Path
import json
from datetime import datetime

class PokemonGameFiles:
    """
    Manages file organization for a Pokemon game.
    Creates a clean directory structure and provides helper methods.
    """
    
    def __init__(self, base_dir: str = 'pokemon_game'):
        self.base = Path(base_dir)
        self.saves = self.base / 'saves'
        self.logs = self.base / 'logs'
        self.data = self.base / 'data'
        self.config = self.base / 'config'
        
        # Create all directories
        for directory in [self.saves, self.logs, self.data, self.config]:
            directory.mkdir(parents=True, exist_ok=True)
    
    def save_game(self, trainer_name: str, game_data: dict) -> Path:
        """Save game data for a trainer."""
        filename = f"{trainer_name.lower()}_save.json"
        save_file = self.saves / filename
        
        game_data['saved_at'] = datetime.now().isoformat()
        save_file.write_text(json.dumps(game_data, indent=2))
        
        print(f"✓ Saved to {save_file}")
        return save_file
    
    def load_game(self, trainer_name: str) -> dict | None:
        """Load game data for a trainer."""
        filename = f"{trainer_name.lower()}_save.json"
        save_file = self.saves / filename
        
        if not save_file.exists():
            print(f"✗ No save file for {trainer_name}")
            return None
        
        data = json.loads(save_file.read_text())
        print(f"✓ Loaded {trainer_name}'s save")
        return data
    
    def list_saves(self) -> list[str]:
        """List all available save files."""
        saves = []
        for save_file in self.saves.glob('*_save.json'):
            trainer = save_file.stem.replace('_save', '').title()
            saves.append(trainer)
        return saves
    
    def log_event(self, message: str) -> None:
        """Append an event to today's log file."""
        today = datetime.now().strftime('%Y-%m-%d')
        log_file = self.logs / f"{today}.log"
        
        timestamp = datetime.now().strftime('%H:%M:%S')
        log_entry = f"[{timestamp}] {message}\n"
        
        # Append to log
        with log_file.open('a') as f:
            f.write(log_entry)

# Test the system
game = PokemonGameFiles()

# Save some games
game.save_game('Ash', {
    'badges': 8,
    'team': ['Pikachu', 'Charizard', 'Blastoise']
})

game.save_game('Misty', {
    'badges': 2,
    'team': ['Starmie', 'Goldeen']
})

# List saves
print(f"\nAvailable saves: {game.list_saves()}")

# Load a game
ash_data = game.load_game('Ash')
if ash_data:
    print(f"Ash's team: {ash_data['team']}")

# Log events
game.log_event('Game started')
game.log_event('Battle won against Brock')
game.log_event('Captured Pikachu')

print("\nGame files structure created successfully!")

---

## **Practice Exercises**

### **Task 1: Create a Path**

Create a Path object for `pokemon/saves/team.json`.

**Expected Output:**
```
pokemon/saves/team.json
```

In [ ]:
# Your code here:


### **Task 2: Check If Exists**

Check if `pokemon_stats.csv` exists.

**Expected Output:**
```
True
```

In [ ]:
# Your code here:


### **Task 3: Create Directory**

Create a directory called `pokemon/saves`.

**Expected Output:**
```
Directory created
```

In [ ]:
# Your code here:


### **Task 4: Get File Parts**

For `pokemon/saves/ash_save.json`, print the parent, name, stem, and suffix.

**Expected Output:**
```
Parent: pokemon/saves
Name: ash_save.json
Stem: ash_save
Suffix: .json
```

In [ ]:
# Your code here:


### **Task 5: Write and Read**

Use `.write_text()` and `.read_text()` with a Path object.

**Expected Output:**
```
Pikachu,Electric,25
```

In [ ]:
# Your code here:


### **Task 6: List Files**

List all `.json` files in `pokemon_game/saves/`.

**Expected Output:**
```
ash_save.json
misty_save.json
...
```

In [ ]:
# Your code here:


### **Task 7: Get File Size**

Get the size of a file in bytes using `.stat()`.

**Expected Output:**
```
42 bytes
```

In [ ]:
# Your code here:


### **Task 8: Current Directory**

Print the current working directory.

**Expected Output:**
```
/home/user/projects/pokemon
```

In [ ]:
# Your code here:


### **Task 9: Delete File**

Create a temp file, then delete it with `.unlink()`.

**Expected Output:**
```
File deleted
```

In [ ]:
# Your code here:


### **Task 10: Build Save System**

Create functions to save and load JSON files in a `saves/` directory.

**Expected Output:**
```
Saved to saves/ash.json
Loaded from saves/ash.json
```

In [ ]:
# Your code here:


---

## **Summary**

- Use `pathlib.Path` for cross-platform file paths (modern, best practice)
- `/` operator joins path components: `Path('a') / 'b' / 'c.txt'`
- `.exists()` — check if path exists
- `.is_file()` — check if it's a file
- `.is_dir()` — check if it's a directory
- `.mkdir(parents=True, exist_ok=True)` — create directories safely
- `.iterdir()` — list all items in a directory
- `.glob('*.json')` — find files matching a pattern
- `.read_text()` / `.write_text()` — quick file I/O
- `.unlink()` — delete a file
- `.stat()` — get file metadata (size, modified time)
- `Path.cwd()` — current working directory
- `Path.home()` — user's home directory

---

## **Quick Reference**

```python
from pathlib import Path

# Create paths
p = Path('folder') / 'subfolder' / 'file.txt'

# Check existence
p.exists()      # Path exists?
p.is_file()     # Is a file?
p.is_dir()      # Is a directory?

# Create directories
p.mkdir(parents=True, exist_ok=True)

# File parts
p.parent        # Parent directory
p.name          # Full filename
p.stem          # Name without extension
p.suffix        # Extension (.txt)

# I/O
p.read_text()           # Read entire file
p.write_text('data')    # Write to file
with p.open('r') as f:  # Or use open()
    ...

# List files
for item in Path('.').iterdir():
    print(item)

for json in Path('.').glob('*.json'):
    print(json)

# Delete
p.unlink()      # Delete file
p.rmdir()       # Delete empty directory

# Info
p.stat().st_size        # File size
Path.cwd()              # Current directory
Path.home()             # Home directory
```